In [1]:
import os
import numpy as np
from dotenv import load_dotenv

from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

## A.　載入 .env 取得 LLM API Key

In [2]:
load_dotenv()
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

C:\Users\PC\AppData\Local\Temp\ipykernel_51496\3523237785.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))


# 

## B.　RAG 實作

### I.　建立向量資料庫

In [3]:
documents = [
    "GPT-4 是 OpenAI 開發的最新語言模型，具備強大的自然語言處理能力。",
    # "RAG（檢索增強生成）結合了檢索與 AI 生成，提升模型回答的準確度。",
    "RAG是知名品牌製作的高效清潔抹布，通常市售30TWD，基本每戶人家都會有一條RAG放置廚房做使用",
    "FAISS 是一款高效的向量搜尋庫，適合用於 AI 相關應用。",
]

In [4]:
# 拆分文本為小塊
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = [Document(page_content=text) for text in text_splitter.split_text("\n".join(documents))]

# 產生嵌入並存入 FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

# 儲存 FAISS 索引
vectorstore.save_local("faiss_index")

print("✅ FAISS 向量資料庫已建立！")

✅ FAISS 向量資料庫已建立！


### II.　使用 Gemini 進行 RAG

In [5]:
### 從 FAISS 檢索最相關的資料
def retrieve_from_faiss(query, top_k=2):
    vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
    relevant_docs = retriever.get_relevant_documents(query)
    return [doc.page_content for doc in relevant_docs]  # 返回最相關的文本

In [6]:
### RAG 流程：檢索 + 生成回答
model = "gpt-4o"

def rag_query(query):
    relevant_docs = retrieve_from_faiss(query)
    context = "\n".join(relevant_docs)

    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 請根據以下資訊回答問題 : {context}
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    res = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    return res.choices[0].message.content

def norm_query(query):
    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    res = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return res.choices[0].message.content

# 

## C.　成果展示

### I.　什麼是 RAG？

In [7]:
norm_query("什麼是 RAG？")

'RAG 是「檢索增強生成」（Retrieval-Augmented Generation）的縮寫。這是一種結合資訊檢索和生成模型的方法，用於改善生成式人工智慧系統的準確性和可靠性。RAG 通常包括以下兩個主要步驟：\n\n1. **資訊檢索**：在這個階段，系統從一個大型資料庫或知識庫中檢索出相關的資料或文件，以作為背景知識或上下文。\n\n2. **生成模型**：使用生成模型（如 GPT-3 等）來生成回應或文字。生成模型會利用先前獲取的相關資訊以提高回答的質量和準確性。\n\n這樣的結合方式使得系統能夠在回答問題時，既有依據地引用檢索到的資料，又能利用生成模型生成流暢且連貫的語言表達。RAG 特別適用於需要高準確性和細節支持的應用情境，比如問答系統。'

In [8]:
rag_query("什麼是 RAG？")

C:\Users\PC\AppData\Local\Temp\ipykernel_51496\3191474142.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)


'RAG 是一個知名品牌，專門製作高效清潔抹布，通常市場售價為 30 新台幣。這種抹布因其高效清潔性能，基本上每戶家庭的廚房裡都會有一條用來做清潔工作。'

### II.　什麼是 Google Gemini？

In [9]:
norm_query("什麼是 Google Gemini？")

'Google Gemini 是由 Google 開發的一個人工智慧（AI）專案。它是 Google 對大型語言模型的一個重要更新，旨在結合各種技術，提升 AI 理解和生成自然語言的能力。Gemini 的目標是通過更有效的數據處理和學習方法，提供更精確、更具上下文的人工智慧回應。它被視為 Google 對抗其他大型語言模型的競爭者，例如 OpenAI 的 GPT 系列。Gemini 努力在自然語言處理、內容生成、搜尋引擎優化等服務中發揮作用。'

In [10]:
rag_query("什麼是 Google Gemini？")

'Google Gemini 是 Google 研發的一個項目，但截至我所接觸到的資料更新範圍，並沒有詳細的官方資訊可說明 Google Gemini 是什麼。可能是新的技術、產品或項目。建議檢查最新的官方公告或新聞稿，以獲取準確的資訊。'

### III.　什麼是 ChatGPT？

In [11]:
norm_query("什麼是 ChatGPT？")

'ChatGPT 是由 OpenAI 開發的一種大型語言模型，基於 GPT-3.5 或更高版本的架構。簡單來說，它是一種使用自然語言處理技術的人工智慧工具，能夠生成類似人類書寫的文本。ChatGPT 可以用於回答問題、撰寫文章、提供建議和進行對話等多種語言相關的應用場景。它通過從大量網路文本資料中學習來提高自身生成文本的能力。需要注意的是，ChatGPT 的知識僅限於它所受訓時的資料，並且不具備即時更新的能力。'

In [12]:
rag_query("什麼是 ChatGPT？")

'ChatGPT 是由 OpenAI 開發的一款基於 GPT 系列模型的聊天機器人應用。它利用了自然語言處理技術，能夠與人類進行對話。ChatGPT 擅長理解並生成類似人類的自然語言回應，常被用於各種對話應用中，如客服、教育、娛樂等。'

### IV.　什麼是 Agent？

In [13]:
norm_query("什麼是 Agent？")

'在計算機科學和人工智慧的領域中，「Agent」指的是一個能夠自主執行行為的計算實體。以下是一些關於 Agent 的基本概念：\n\n1. **自主性**：Agent 具備自主決策的能力，能夠依據所處的環境和接收到的資訊決定接下來的行動。\n\n2. **感知與行動**：Agent 通常具有感知環境的能力（通過感測器）和對環境進行回應的能力（通過效應器）。\n\n3. **持續運行**：Agent 通常是長時間運行的系統，保持對環境的持續觀察並作出適當反應。\n\n4. **目標導向**：Agent 會根據預設的目標或任務來致力於達成某些結果。例如，搜尋引擎中的網路爬蟲可以視為一種 Agent，其目的是收集網頁資訊。\n\n5. **互動性**：很多 Agent 能與其他 Agent 或使用者進行互動，交換信息或協同完成任務。\n\n在現代應用中， Agent 的概念被廣泛應用於軟體代理、智慧家庭裝置、遊戲 AI、中介服務等多個領域。'

In [14]:
rag_query("什麼是 Agent？")

'"Agent" 在多個領域中具有不同的含義：\n\n1. **人工智慧（AI）中的 Agent**：在人工智慧和機器學習中，Agent 通常指可以感知環境並採取行動的系統或程式。AI Agent 可以是軟體或硬體，通過分析輸入作出決策或執行特定任務。\n\n2. **軟體工程中的 Agent**：在軟體工程中，Agent 可能指一個獨立的程式或進程，能夠自動執行某些任務，通常需要在特定的系統環境下運行。\n\n3. **一般詞義中的 Agent**：Agent 也可以泛指任何代表他人或某一組織執行職責的人，如房地產代理、運動經紀人等。\n\n根據上下文，"Agent" 可以有多種解釋，因此重要的是要理解其在特定情境中的具體含義。'

### V.　RAG 是抹布嗎? 超商可以買到它嗎?

In [15]:
norm_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'"RAG" 在日常用語中通常指的是抹布，是用來清潔物品或表面的工具。超商通常會出售這些清潔用品，包括各種用途和材料的抹布，因此你可能可以在超商找到抹布。不過，具體的供應情況可能會因地區和超商的品牌而有所不同，建議可以到附近的超商查看清潔用品區來確認。'

In [16]:
rag_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'根據你提供的資訊，RAG 是知名品牌製作的高效清潔抹布。一般家庭都會在廚房放置一條用來清潔。至於是否能在超商買到它，雖然並未明確指出超商有售，但既然是一種普通的消耗品，具有一定的市場需求，所以在許多超商或其他零售商店有販售的可能性較高。不過，具體可購買的地點可能會因地區和超商的產品供應情況而有所不同。'